In [1]:
import pymysql  
import  pymysql.cursors 
#https://www.runoob.com/python/python-mysql.html

In [2]:
from sqlalchemy import create_engine
import pandas as pd

In [3]:
#https://www.runoob.com/python3/python3-mysql.html
import pymysql
# 打开数据库连接
db = pymysql.connect("118.190.201.131","xxx","xxxx","xxxxx" ) #,"TESTDB" 
# 使用 cursor() 方法创建一个游标对象 cursor
cursor = db.cursor()
# 使用 execute() 方法执行 SQL，如果表存在则删除
sql_2="SHOW TABLES"
cursor.execute(sql_2)
result2=cursor.fetchall()
print('result2:',result2) 
# 关闭数据库连接
db.close()

result2: (('dim_category',), ('dim_date',), ('dim_goods',), ('dim_member',), ('dim_shop',), ('fct_sales',), ('fct_sales_item',))


In [36]:
from sqlalchemy import create_engine
import pandas as pd
engine=create_engine('mysql+pymysql://账户名:密码@IP:3306/dw')
# 查询语句，选出employee表中的所有数据
sql = "SELECT * FROM fct_sales limit 8" 
# read_sql_query的两个参数: sql语句， 数据库连接
df_fct_sales = pd.read_sql_query(sql, engine)
# 输出employee表的查询结果
(df_fct_sales.head(3))

,salesID,salesNo,dimDateID,dimShopID,dimMemberID,QTY,AMT,pAMT,mpAMT,updateTime
0,1,34131002542176,20170602,33,4821,1.000,7.99,0.0,0.210,2017-09-09 10:24:16
1,2,34131002542177,20170602,34,1730,1.000,227.00,0.0,12.000,2017-09-09 10:24:16
2,3,34131002542178,20170602,33,513,5.046,75.64,0.0,3.022,2017-09-09 10:24:16


In [35]:
#计算7月某个周的每个会员订单数，以及订单比
#第一步  各个会员订单数
from sqlalchemy import create_engine
import pandas as pd
engine=create_engine('mysql+pymysql://账户名:密码@IP:3306/dw')
# 查询语句，选出employee表中的所有数据
sql = """SELECT dimMemberID,COUNT(AMT) as order_number
        FROM fct_sales  where dimDateID between 20170602 and 20170609 
        group by dimMemberID  """
df_fct_sales = pd.read_sql_query(sql, engine)
df_fct_sales.head(3)  

,dimMemberID,order_number
0,4821,3
1,1730,2
2,513,19


In [51]:
#第二步
#计算总和
sql = """select sum(order_number) as sum_all_orders
        from
            (SELECT dimMemberID,COUNT(AMT) as order_number
            FROM fct_sales  where dimDateID between 20170602 and 20170609   and dimMemberID<>0
            group by dimMemberID) as temp
        
"""

df_fct_sales = pd.read_sql_query(sql, engine)
df_fct_sales.head(3)  

,sum_all_orders
0,14912.0


In [54]:
#第三步 
sql="""select dimMemberID,order_number,order_number/(select sum(order_number) as sum_all_orders
        from
            (SELECT dimMemberID,COUNT(AMT) as order_number
            FROM fct_sales  where dimDateID between 20170602 and 20170609 and dimMemberID<>0
            group by dimMemberID) as temp) as rate
        from
            (SELECT dimMemberID,COUNT(AMT) as order_number
            FROM fct_sales  where dimDateID between 20170602 and 20170609 and dimMemberID<>0
            group by dimMemberID) as temp
        
"""
df_fct_sales = pd.read_sql_query(sql, engine)
df_fct_sales.head(3)  

,dimMemberID,order_number,rate
0,4821,3,0.0002
1,1730,2,0.0001
2,513,19,0.0013


In [55]:
#第三步或者 
sql="""select dimMemberID,order_number,order_number/14912 as rate
        from
            (SELECT dimMemberID,COUNT(AMT) as order_number
            FROM fct_sales  where dimDateID between 20170602 and 20170609 
            group by dimMemberID) as temp
        
"""
df_fct_sales = pd.read_sql_query(sql, engine)
df_fct_sales.head(3)  

,dimMemberID,order_number,rate
0,4821,3,0.0002
1,1730,2,0.0001
2,513,19,0.0013


In [7]:
from sqlalchemy import create_engine
import pandas as pd
engine=create_engine('mysql+pymysql://账户名:密码@IP:3306/dw')
# 查询语句，选出employee表中的所有数据
sql = "SELECT * FROM dim_member" 
# read_sql_query的两个参数: sql语句， 数据库连接
df = pd.read_sql_query(sql, engine)
# 输出employee表的查询结果
print(df)

       dimMemberID     memberCode  name  birthDate  registerDate  \
0                2  5072121034016  None   19520217      20130131   
1                3  5072125386012  None   19420213      20130130   
2                4  5072126676013  None   19390209      20130122   
3                5  5341080159000  None   19560213      20130124   
4                6  5341080164004  None   19500210      20130125   
5                7  5341080179008  None   19690225      20130206   
6                8  5341080197002  None   19820203      20130116   
7                9  5341080252008  None   19900202      20130129   
8               10  5341080267002  None   19920204      20130108   
9               11  5341080292004  None   19800205      20130201   
10              12  5341080297009  None   20080206      20130206   
11              13  5341080310005  None   19910209      20130122   
12              14  5341080314003  None   19170205      20130107   
13              15  5341080354009  None   200002

In [8]:
df.columns

Index(['dimMemberID', 'memberCode', 'name', 'birthDate', 'registerDate',
       'firstShoppingDate', 'lastShoppingDate'],
      dtype='object')

In [5]:
df.to_csv('零售.csv',encoding='utf_8_sig')

In [7]:
from sqlalchemy import create_engine
import pandas as pd
engine=create_engine('mysql+pymysql://账户名:密码@IP:3306/dw')
# 查询语句，选出employee表中的所有数据
sql = "SELECT * FROM dim_shop" 
# read_sql_query的两个参数: sql语句， 数据库连接
df = pd.read_sql_query(sql, engine)
# 输出employee表的查询结果
print(df)

   dimShopID shopCode name areaCode areaName
0         33     HD01  广场店   HDOOOO     华东大区
1         34     BJ01  花桥店   BJOOOO     北京大区
2         35     HD02  上海店   HDOOOO     华东大区
3         36   BJ0002  东城店   BJOOOO     北京大区


In [9]:
sql = "show tables" 
cursor.execute(sql)
results = cursor.fetchall()
print(results)

(('dim_category',), ('dim_date',), ('dim_goods',), ('dim_member',), ('dim_shop',), ('fct_sales',), ('fct_sales_item',))


In [25]:
import pymysql# 打开数据库连接
db = pymysql.connect("ip","账户名","密码","库名" )
# 使用cursor()方法获取操作游标 
cursor = db.cursor()
# SQL 查询语句
sql = "SHOW COLUMNS FROM dim_goods"
try:
    cursor.execute(sql)
    #results = cursor.fetchone()
    results = cursor.fetchmany(6)
    #print(results)
    for i in results:
        print(i,"\t")
except:
    print ("Error: unable to fetch data")
# 关闭数据库连接
db.close()

('dimGoodsID', 'int(11)', 'NO', 'PRI', None, '') 	
('categoryID1', 'tinyint(11)', 'YES', '', None, '') 	
('categoryName1', 'varchar(255)', 'YES', '', None, '') 	
('categoryID2', 'tinyint(4)', 'YES', '', None, '') 	
('categoryName2', 'varchar(255)', 'YES', '', None, '') 	
('categoryID3', 'tinyint(4)', 'YES', '', None, '') 	


In [11]:
sql = "SELECT dimGoodsID FROM dim_goods" 
cursor.execute(sql)
results = cursor.fetchall()
print(results)

((400001,), (1010023,), (1010030,), (1010031,), (1010033,), (1010044,), (1010064,), (1010068,), (1010069,), (1010072,), (1010079,), (1010093,), (1010100,), (1010107,), (1010127,), (1010132,), (1010137,), (1010151,), (1010160,), (1010194,), (1010198,), (1010208,), (1010223,), (1010224,), (1010225,), (1010226,), (1010236,), (1010249,), (1010273,), (1010280,), (1010281,), (1010325,), (1010349,), (1010350,), (1010351,), (1010356,), (1010363,), (1010367,), (1010388,), (1010392,), (1010396,), (1010398,), (1010400,), (1010409,), (1010416,), (1010422,), (1010423,), (1010424,), (1010427,), (1010428,), (1010433,), (1010441,), (1010445,), (1010446,), (1010451,), (1010459,), (1010461,), (1010499,), (1010502,), (1010504,), (1010513,), (1110006,), (1110016,), (1110027,), (1110032,), (1110033,), (1110161,), (1110218,), (1110220,), (1110238,), (1110271,), (1110273,), (1110276,), (1110279,), (1110299,), (1110300,), (1110316,), (1110317,), (1110318,), (1110332,), (1110333,), (1110365,), (1110372,), (111

In [10]:
# 数据更新操作
#!/usr/bin/python3
import pymysql
# 打开数据库连接
db = pymysql.connect("localhost","root","deepwind","demo" )
# 使用cursor()方法获取操作游标 
cursor = db.cursor()
# SQL 更新语句
sql = "UPDATE EMPLOYEE SET AGE = AGE + 1 WHERE SEX = '%c'" % ('M')
try:
   # 执行SQL语句
   cursor.execute(sql)
   # 提交到数据库执行
   db.commit()
except:
   # 发生错误时回滚
   db.rollback()
# 关闭数据库连接
db.close()       

In [14]:
import pymysql
def ab_data(array,sts,dbname):
    """获取样例的异常时刻数据"""
    if len(str(round(float(array[2])))) > 8:
        table = str(array[2])[0:8] + '_' + str(array[2])[8]
    else:
        table = str(round(float(array[2])))
    st = str(array[5] + datetime.timedelta(minutes= -7))
    et = str(array[6] + datetime.timedelta(minutes= 3))
    conn = pymysql.connect(host='192.168.0.36', user='zcr', passwd='zcr123.', db=dbname, port=5029, charset='utf8')
    cur = conn.cursor()
    sql = "SELECT  "+sts+" FROM t" +table+"_all WHERE real_time BETWEEN '" + st + "' AND  '" + et + "' "
    # print(sql)
    cur.execute(sql)
    datas = np.array(cur.fetchall())
    return(datas)
# 明阳  conn = pymysql.connect(host='192.168.0.19', user='zcr', passwd='zcr123.', db='iot_wind', port=3306, charset='utf8')

#https://blog.csdn.net/u011583336/article/details/80999043  报错解决方案
import mysql.connector
mydb = mysql.connector.connect(host="localhost",user="root",passwd="deepwind")
mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE runoob_db")


In [3]:
#!/usr/bin/env python
# -*-encoding:utf-8-*-
# https://blog.csdn.net/ATOOHOO/article/details/88173151    pymysql单条插入数据和批量插入数据：
import pymysql
# 打开数据库连接
db = pymysql.connect("localhost","root","deepwind","demo")
# 使用 cursor() 方法创建一个游标对象 cursor
cursor = db.cursor()
# SQL 插入语句
sql = "INSERT INTO EMPLOYEE(FIRST_NAME, \
       LAST_NAME, AGE, SEX, INCOME) \
       VALUES (%s,%s,%s,%s,%s)"
# 区别与单条插入数据，VALUES ('%s', '%s',  %s,  '%s', %s) 里面不用引号
 
val = (('li', 'si', 16, 'F', 1000),
       ('Bruse', 'Jerry', 30, 'F', 3000),
       ('Lee', 'Tomcat', 40, 'M', 4000),
       ('zhang', 'san', 18, 'M', 1500))
try:
   # 执行sql语句
   cursor.executemany(sql,val)
   # 提交到数据库执行
   db.commit()
except:
   # 如果发生错误则回滚
   db.rollback()
 
# 关闭数据库连接
db.close()